In [21]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))
display(HTML("<style>.container {width:100% !important; }</style>"))

# check environment
import os
print(f'Conda Environment: ' + os.environ['CONDA_DEFAULT_ENV'])

Conda Environment: roicat


This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server
import bnpm

QObject::moveToThread: Current thread (0x1bcbc90) is not the object's thread (0x1deb370).
Cannot move to target thread (0x1bcbc90)

QObject::moveToThread: Current thread (0x1bcbc90) is not the object's thread (0x1deb370).
Cannot move to target thread (0x1bcbc90)

QObject::moveToThread: Current thread (0x1bcbc90) is not the object's thread (0x1deb370).
Cannot move to target thread (0x1bcbc90)

QObject::moveToThread: Current thread (0x1bcbc90) is not the object's thread (0x1deb370).
Cannot move to target thread (0x1bcbc90)

QObject::moveToThread: Current thread (0x1bcbc90) is not the object's thread (0x1deb370).
Cannot move to target thread (0x1bcbc90)

QObject::moveToThread: Current thread (0x1bcbc90) is not the object's thread (0x1deb370).
Cannot move to target thread (0x1bcbc90)

QObject::moveToThread: Current thread (0x1bcbc90) is not the object's thread (0x1deb370).
Cannot move to target thread (0x1bcbc90)

QObject::moveToThread: Current thread (0x1bcbc90) is not the object's thread

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username:  rh183
Password:  ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Batch run

In [12]:
mouse = 'mouse_0403R'

In [13]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path=f'/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/{mouse}/',
    search_pattern_re='cam4.*avi',
    max_depth=5,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230708/cam42023-07-08T12_17_59.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230709/cam42023-07-09T12_19_41.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230702/cam42023-07-02T14_46_45.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230703/cam42023-07-03T11_31_50.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230707/cam42023-07-07T12_09_13.avi
/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230706/cam42023-07-06T13_45_37.avi


In [14]:
dates_toUse = natsort.natsorted([Path(p).parts[-2] for p in paths_found])
dates_toUse

['20230702', '20230703', '20230706', '20230707', '20230708', '20230709']

In [15]:
paths_vids = {d: v for d, v in zip(dates_toUse, paths_found)}

## Start a small interactive node

In [16]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash');

bash --pty -p interactive -t 0-00:30:00 --mem=4G /bin/ 

srun: job 31497834 queued and waiting for resources



## Pull/update repo

In [17]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
send_expect('git pull');

cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP

git pull

srun: job 31497834 has been allocated resources

cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP
git pull

(base) [rh183@compute-a-16-170 ~]$ 


## Activate environment

In [18]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

_repos/NBAP/hms/neurobio/sabatini/rich/github 
(base) [rh183@compute-a-16-170 NBAP]$ git pull



## Prepare commands

In [19]:
# duration_sessions = {date: 60*60 if float(date) < float(20230725) else 80*60 for date in dates_toUse}

duration_sessions = {date: 60*60 for date in dates_toUse}

In [26]:
for path_vid in paths_found[:]:

    mouse = Path(path_vid).parts[-4]
    date = Path(path_vid).parts[-2]
    
    name_slurm = mouse[-3:] + '_' + date[-4:]

    path_vid = path_vid
    path_mask = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction') / mouse / 'masks' / date / ('mask_' + Path(path_vid).stem + '.npy') )

    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction') / mouse / date )

    name_job = 'jobNum_'


    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env

    time_fastForward = duration_sessions[date] - 1
    
    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {path_vid} \
    {path_mask} \
    {name_job} \
    {name_slurm} \
    {name_env}\
    {time_fastForward}\
    ",
    }
    display(commands)
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    send_expect(commands['dispatch']);

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230702/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230702     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230702/cam42023-07-02T14_46_45.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230702/mask_cam42023-07-02T14_46_45.npy     jobNum_     03R_0702     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

 3599        03R_0709     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR
Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py to /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230709/extract_eye_brightness_trace.py
Submitting job: jobNum_ 0
Submitted batch job 31498829
AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-170 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230703/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230703     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230703/cam42023-07-03T11_31_50.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230703/mask_cam42023-07-03T11_31_50.npy     jobNum_     03R_0703     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

 3599        03R_0702     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR
    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230703/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230703     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230703/cam42023-07-03T11_31_50.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230703/mask_cam42023-07-03T11_31_50.npy     jobNum_     03R_0703     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230706/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230706     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230706/cam42023-07-06T13_45_37.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230706/mask_cam42023-07-06T13_45_37.npy     jobNum_     03R_0706     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

...

 3599    /FR    
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230707/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230707     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230707/cam42023-07-07T12_09_13.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230707/mask_cam42023-07-07T12_09_13.npy     jobNum_     03R_0707     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py to /n/data1/hms/neurobio/sabatin

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230708/cam42023-07-08T12_17_59.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230708/mask_cam42023-07-08T12_17_59.npy     jobNum_     03R_0708     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

 3599        03R_0707     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    
dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230708/cam42023-07-08T12_17_59.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230708/mask_cam42023-07-08T12_17_59.npy     jobNum_     03R_0708     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230709/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230709     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230709/cam42023-07-09T12_19_41.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230709/mask_cam42023-07-09T12_19_41.npy     jobNum_     03R_0709     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    '}

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230709/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230709     /n/data1/hms/neurobio/sabatini/rich/data/res2p/round_7_experiments/mouse_0403R/camera_data/20230709/cam42023-07-09T12_19_41.avi     /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230709/mask_cam42023-07-09T12_19_41.npy     jobNum_     03R_0709     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    3599    

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/4a3_analyze_eye_laser_trace/extract_eye_brightness_trace.py to /n/data1/hms/neurobio/sabatini/rich/analysis/e

### check on job

### Check if it worked

In [27]:
test = natsort.natsorted(sftp.search_recursive(
    path=f'/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/{mouse}',
    search_pattern_re=r'idx_eye_laser.pkl',
))
display(test)

/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230709/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230702/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230707/jobNum_0/idx_eye_laser.pkl
/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230706/jobNum_0/idx_eye_laser.pkl


['/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230702/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230706/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230707/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708/jobNum_0/idx_eye_laser.pkl',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230709/jobNum_0/idx_eye_laser.pkl']

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()